# COURSE: A deep understanding of deep learning
## SECTION: Measuring model performance
### LECTURE: Computation time
#### TEACHER: Mike X Cohen, sincxpress.com
##### COURSE URL: udemy.com/course/deeplearning_x/?couponCode=202401

In [1]:
# import libraries
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

# New!
import time
# Well, OK, it's not really new. See, e.g., DUDL_metaparams_CodeChallengeBatches

import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

# Import and process the data

In [2]:
# import dataset (comes with colab!)
data = np.loadtxt(open('sample_data/mnist_train_small.csv','rb'),delimiter=',')

# extract labels (number IDs) and remove from data
labels = data[:,0]
data   = data[:,1:]

# normalize the data to a range of [0 1]
dataNorm = data / np.max(data)

# Create train/test groups using DataLoader

In [3]:
# Step 1: convert to tensor
dataT   = torch.tensor( dataNorm ).float()
labelsT = torch.tensor( labels ).long()

# Step 2: use scikitlearn to split the data
train_data,test_data, train_labels,test_labels = train_test_split(dataT, labelsT, test_size=.1)


# Step 3: convert into PyTorch Datasets
train_data = torch.utils.data.TensorDataset(train_data,train_labels)
test_data  = torch.utils.data.TensorDataset(test_data,test_labels)

# Step 4: translate into dataloader objects
batchsize  = 32
train_loader = DataLoader(train_data,batch_size=batchsize,shuffle=True,drop_last=True)
test_loader  = DataLoader(test_data,batch_size=test_data.tensors[0].shape[0])

# Create the DL model

In [4]:
# create a class for the model
def createTheMNISTNet():

  class mnistNet(nn.Module):
    def __init__(self):
      super().__init__()

      ### input layer
      self.input = nn.Linear(784,64)
      
      ### hidden layer
      self.fc1 = nn.Linear(64,32)
      self.fc2 = nn.Linear(32,32)

      ### output layer
      self.output = nn.Linear(32,10)

    # forward pass
    def forward(self,x):
      x = F.relu( self.input(x) )
      x = F.relu( self.fc1(x) )
      x = F.relu( self.fc2(x) )
      return self.output(x)
  
  # create the model instance
  net = mnistNet()
  
  # loss function
  lossfun = nn.CrossEntropyLoss()

  # optimizer
  optimizer = torch.optim.Adam(net.parameters(),lr=.01)

  return net,lossfun,optimizer

# Create a function that trains the model

In [5]:
def function2trainTheModel():

  # Start the timer!
  timerInFunction = time.process_time()

  # number of epochs
  numepochs = 10
  
  # create a new model
  net,lossfun,optimizer = createTheMNISTNet()

  # initialize losses
  losses    = torch.zeros(numepochs)
  trainAcc  = []
  testAcc   = []


  # loop over epochs
  for epochi in range(numepochs):

    # loop over training data batches
    batchAcc  = []
    batchLoss = []
    for X,y in train_loader:

      # forward pass and loss
      yHat = net(X)
      loss = lossfun(yHat,y)

      # backprop
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # loss from this batch
      batchLoss.append(loss.item())

      # compute accuracy
      matches = torch.argmax(yHat,axis=1) == y     # booleans (false/true)
      matchesNumeric = matches.float()             # convert to numbers (0/1)
      accuracyPct = 100*torch.mean(matchesNumeric) # average and x100
      batchAcc.append( accuracyPct )               # add to list of accuracies
    # end of batch loop...

    # now that we've trained through the batches, get their average training accuracy
    trainAcc.append( np.mean(batchAcc) )

    # and get average losses across the batches
    losses[epochi] = np.mean(batchLoss)

    # test accuracy
    X,y = next(iter(test_loader)) # extract X,y from test dataloader
    with torch.no_grad(): # deactivates autograd
      yHat = net(X)
      
    # compare the following really long line of code to the training accuracy lines
    testAcc.append( 100*torch.mean((torch.argmax(yHat,axis=1)==y).float()) )

    # Finally, report the epoch number, computation time, and accuracy
    comptime = time.process_time() - timerInFunction
    print(f'Epoch {epochi+1}/{numepochs}, elapsed time: {comptime:.2f} sec, test accuracy {testAcc[-1]:.0f}%')

  # end epochs

  # function output
  return trainAcc,testAcc,losses,net

# Run the model and show the results!

In [6]:
trainAcc,testAcc,losses,net = function2trainTheModel()

Epoch 1/10, elapsed time: 5.51 sec, test accuracy 93%
Epoch 2/10, elapsed time: 11.05 sec, test accuracy 93%
Epoch 3/10, elapsed time: 16.43 sec, test accuracy 93%
Epoch 4/10, elapsed time: 21.79 sec, test accuracy 95%
Epoch 5/10, elapsed time: 27.52 sec, test accuracy 94%
Epoch 6/10, elapsed time: 32.87 sec, test accuracy 94%
Epoch 7/10, elapsed time: 44.13 sec, test accuracy 95%
Epoch 8/10, elapsed time: 58.84 sec, test accuracy 95%
Epoch 9/10, elapsed time: 73.71 sec, test accuracy 95%
Epoch 10/10, elapsed time: 88.92 sec, test accuracy 94%


In [7]:
# now run a second timer over repeated iterations

# Start the timer! (note the different variable name)
timerOutsideFunction = time.process_time()

for i in range(10):
  function2trainTheModel()

TotalExperimentTime = time.process_time() - timerOutsideFunction
print(f'\n\n\nTotal elapsed experiment time: {TotalExperimentTime/60:.2f} minutes')

Epoch 1/10, elapsed time: 10.18 sec, test accuracy 89%
Epoch 2/10, elapsed time: 21.90 sec, test accuracy 93%
Epoch 3/10, elapsed time: 34.68 sec, test accuracy 92%
Epoch 4/10, elapsed time: 47.49 sec, test accuracy 93%
Epoch 5/10, elapsed time: 60.59 sec, test accuracy 94%
Epoch 6/10, elapsed time: 74.06 sec, test accuracy 91%
Epoch 7/10, elapsed time: 87.22 sec, test accuracy 95%
Epoch 8/10, elapsed time: 100.24 sec, test accuracy 95%
Epoch 9/10, elapsed time: 106.89 sec, test accuracy 95%
Epoch 10/10, elapsed time: 120.84 sec, test accuracy 96%
Epoch 1/10, elapsed time: 10.23 sec, test accuracy 93%
Epoch 2/10, elapsed time: 22.87 sec, test accuracy 94%
Epoch 3/10, elapsed time: 36.75 sec, test accuracy 93%
Epoch 4/10, elapsed time: 43.91 sec, test accuracy 93%
Epoch 5/10, elapsed time: 49.33 sec, test accuracy 94%
Epoch 6/10, elapsed time: 54.91 sec, test accuracy 94%
Epoch 7/10, elapsed time: 60.42 sec, test accuracy 94%
Epoch 8/10, elapsed time: 66.94 sec, test accuracy 93%
Epoch 

# Additional explorations

In [ ]:
# 1) Modify the TotalExperimentTime code so that it prints minutes and seconds. For example, 500 seconds is 
#    8 minutes and 20 seconds.
# 
# 2) Modify the code inside the training function so that the display prints on only every 5th epoch.
# 